**disallowedSynonym.ipynb**

narrowSynonym or broadSynonym defined in Format subontology.

**Documentation:** https://github.com/edamontology/edamverify/blob/master/docs/disallowedSynonym.md

Set constants for script return values. Load EDAM_dev.owl from GitHub into an RDF graph.

In [1]:
import os
from rdflib import ConjunctiveGraph, Namespace
import json
from collections import OrderedDict

# Constants for script error reporting as per https://github.com/edamontology/edamverify.
NOERR = "NOERR"
INFO  = "INFO"
WARN  = "WARN"
ERROR = "ERROR"

#Load EDAM_dev.owl from GitHub into an RDF graph.
print("Loading graph ...", end="")
g = ConjunctiveGraph()
g.load(os.environ.get('EDAM_PATH', '../EDAM_dev.owl'), format='xml')
# g.load('https://raw.githubusercontent.com/edamontology/edamontology/master/EDAM_dev.owl', format='xml')
# g.load('EDAM_dev.owl')
g.bind('edam', Namespace('http://edamontology.org#'))
print("done!")

Loading graph ...done!


https://neuroml.org/neuromlv2|https://neuroml-db.org/ does not look like a valid URI, trying to serialize this will break.


Define SPARQL query to extract ID, term, and (if available) narrow and broad synonyms of all Format concepts. Run the query.

In [2]:
# Compile SPARQL query
query_term = """
SELECT ?id ?term ?narrow_syn ?broad_syn WHERE
{
?id rdfs:label ?term .
OPTIONAL {?id oboInOwl:hasNarrowSynonym ?narrow_syn .}
OPTIONAL {?id oboInOwl:hasBroadSynonym ?broad_syn .}
FILTER regex(str(?id), "format_")
}
"""

# Run SPARQL query and collate results
errfound = False    
report = list()
results = g.query(query_term)

Analyse results of query.

**NB:**
**CAUTION:** If a (part of a) SPARQL query cannot be satisfied then the associated SPARQL variable will get assigned the value of "None" - *note* "None" as a *string value*.  This should not be confused with the **None** (null) Python keyword. 



In [3]:
report.append("Synonym (disallowed) defined for these Format concepts:")

for r in results :
    
    id     = str(r['id'])
    term   = str(r['term'])
    narrow_syn = str(r['narrow_syn']) 
    broad_syn = str(r['broad_syn']) 
            
#    print(str(r['id']), "   ", id, "   narrow:", narrow_syn, "   broad:", broad_syn)
   
    # "None" string is return from SPARQL query where value was not found
    if narrow_syn != "None":
        errfound = True
        report.append("Disallowed narrowSynonym ::: " + id +  ' (' + term + ')' + " ::: " + "narrowSynonym:" + "   " + narrow_syn)

    if broad_syn != "None":
        errfound = True
        report.append("Disallowed broadSnonym ::: " + id +  ' (' + term + ')' + " ::: " + "broadSynonym:" + "   " + broad_syn)

Write report and return appropriate value.

In [4]:
# Write report
# Use ordered dictionary to get consistent listing of properties in the JSON output
report_dict = OrderedDict

if errfound:
    report_dict = {
        'test_name': 'disallowedSynonym',
        'status' : ERROR,
        'reason' : report
    }
else:
    report_dict = {
        'test_name': 'disallowedSynonym',
        'status' : NOERR,
        'reason' : '-'
    }

report_json = json.dumps(report_dict, indent=4)
print(report_json)

{
    "test_name": "disallowedSynonym",
    "status": "ERROR",
    "reason": [
        "Synonym (disallowed) defined for these Format concepts:",
        "Disallowed narrowSynonym ::: http://edamontology.org/format_2330 (Textual format) ::: narrowSynonym:   txt",
        "Disallowed narrowSynonym ::: http://edamontology.org/format_2330 (Textual format) ::: narrowSynonym:   Plain text format",
        "Disallowed narrowSynonym ::: http://edamontology.org/format_3865 (RNA annotation format) ::: narrowSynonym:   miRNA data format",
        "Disallowed broadSnonym ::: http://edamontology.org/format_3865 (RNA annotation format) ::: broadSynonym:   RNA data format",
        "Disallowed narrowSynonym ::: http://edamontology.org/format_3865 (RNA annotation format) ::: narrowSynonym:   microRNA data format",
        "Disallowed broadSnonym ::: http://edamontology.org/format_3865 (RNA annotation format) ::: broadSynonym:   RNA data format",
        "Disallowed broadSnonym ::: http://edamontology